In [1]:
# PU ExtraTree - A DT Classifier for PU Learning
import numpy as np
import scipy.stats
import scipy.sparse

class PUExtraTree:
    def __init__(self, risk_estimator = "nnPU",
                 loss = "quadratic",
                 max_depth = None,
                 min_samples_leaf = 1,
                 max_features = "sqrt",
                 max_candidates = 1):

        """

        Parameters
        ----------
        risk_estimator : {"PN", "uPU", "nnPU"}, default='nnPU'
            PU data based risk estimator. Supports supervised (PN) learning, unbiased PU (uPU) learning and nonnegative PU (nnPU) learning.
        loss : {"quadratic", "logistic"}, default='quadratic'
            The function to measure the cost of making an incorrect prediction. Supported loss functions are:
            "quadratic" l(v,y) = (1-vy)^2 and
            "logistic" l(v,y) = ln(1+exp(-vy)).
        max_depth : int or None, default=None
            The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_leaf samples.
        min_samples_leaf : int, default=1
            The minimum number of samples required to be at a leaf node. The default is 1.
        max_features : int or {"sqrt", "all"}, default="sqrt"
            The number of features to consider when looking for the best split. If "sqrt", then max_features = ceil(sqrt(n_features)). If "all", then max_features = n_features.
        max_candidates : int, default=1
            Number of randomly chosen split points to consider for each candidate feature.

        Returns
        -------
        None.

        """

        self.risk_estimator = risk_estimator
        self.loss = loss
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.max_candidates = max_candidates

        self.is_trained = False # indicate if tree empty/trained
        self.leaf_count = 0
        self.current_max_depth = 0
        self.nodes = {(0,0): {'data': None, 'j': None, 'xi': None, 'g': None,
                              'is_leaf': None, 'risk_reduction': None}}


    def create_successor(self, node, side):
        """
        Create an empty child node (either T or F) in the tree.
        Parameters
        ----------
        node : tuple of length 2.
            The parent node. First element is the depth in the tree, second element is the position at that depth.
        side : {"T", "F"} or {"L", "R"}
            Whether the node corresponds to a True or False split.
        Returns
        -------
        None.

        """

        row, column = node
        if side in ['T','L']:
            self.nodes[(row+1, 2*column)] = {'data': None, 'j': None,
                                                   'xi': None, 'g': None,
                                                   'is_leaf': None, 'loss': None,
                                                   'risk_reduction': None}
        elif side in ['F','R']:
            self.nodes[(row+1, 2*column+1)] = {'data': None, 'j': None,
                                                     'xi': None, 'g': None,
                                                     'is_leaf': None, 'loss': None,
                                                     'risk_reduction': None}
        elif side not in ['T','F','L','R']:
            print('choose valid position of child node: \'L\', \'R\', \'T\', \'F\'')



    def get_parent(self, node, return_truth_val):
        """
        Return parent node, and optionally the relationship to child node (T/F).

        Parameters
        ----------
        node : tuple of length 2
            The child node.
        return_truth_val : bool
            Indicate whether the truth value should also be returned, that is, whether the child node corresponds to a true or false split.
        Returns
        -------
        tuple of length 2 or (tuple of length 2, bool)
            The parent node, optionally the relationships to the parent nodes.

        """

        parent = (node[0] - 1, node[1] // 2)
        if return_truth_val:
            if node[1] % 2 == 0:
                return parent, True
            else:
                return parent, False
        else:
            return parent



    def get_ancestory(self, node):
        """
        Get parent nodes and relationship to the child nodes all the way to the root.

        Parameters
        ----------
        node : tuple of length 2
            Child node.

        Returns
        -------
        list
            List of nodes.
        bools : list
            List of bools with the relationships to the parents.

        """
        chain = [node]
        bools = []
        while chain[-1] != (0,0):
            parent, relationship = self.get_parent(chain[-1], True)
            chain += [parent]
            bools += [relationship]

        return chain[1:], bools


    def load_tree(self, nodes):
        """
        Load saved tree.

        Parameters
        ----------
        nodes : dictionary
            Dictionary describing the trained decision tree. Typically output from self.nodes.

        Returns
        -------
        None.

        """

        self.nodes = nodes
        self.is_trained = True




    def fit(self, pi, P = None, U = None, N = None):
        """
        Fit the decision tree.

        Parameters
        ----------
        pi : float
            Prior probability that an example belongs to the positive class.
        P : array-like of shape (n_p, n_features), default=None
            Training samples from the positive class.
        U : array-like of shape (n_u, n_features), default=None
            Unlabeled training samples.
        N : array-like of shape (n_n, n_features), default=None
            Training samples from the negative class if performing PN learning.

        Returns
        -------
        self
            Returns instance of self.

        """
        if self.risk_estimator in ['uPU', 'nnPU']:
            X = np.concatenate((P, U), axis = 0)
            y = np.concatenate((np.ones(len(P)), np.zeros(len(U))))
        elif self.risk_estimator in ['PN']:
            X = np.concatenate((P, N), axis = 0)
            y = np.concatenate((np.ones(len(P)), -np.ones(len(N))))

        # X = X.astype(np.float32)
        y = y.astype(np.int8).flatten()
        n, self.d = X.shape
        n_p = (y == 1).sum()
        n_u = (y == 0).sum()
        n_n = (y == -1).sum()
        self.pi = pi

        if self.pi is None:
            print('please specify pi')

        if self.max_features == 'sqrt':
            self.max_features = int(np.ceil(np.sqrt(X.shape[1])))
        elif self.max_features == 'all':
            self.max_features = X.shape[1]
        elif self.max_features in [i for i in range(1, self.d+1)]:
            None
        else:
            print('select valid number of max features to consider splitting on.')
            return None


        self.nodes[(0,0)]['data'] = scipy.sparse.coo_matrix(np.ones(n).astype(bool))

        def data_at_node(node):
            # return subset of training data in partition specified by certain node

            if self.nodes[node]['data'] is not None:
                return self.nodes[node]['data'].toarray()[0]
            else:
                # get indices of data at parent
                parent_node, relationship = self.get_parent(node, True)
                ind_parent = self.nodes[parent_node]['data'].toarray()[0].copy()
                checks = (X[ind_parent, self.nodes[parent_node]['j']] <= self.nodes[parent_node]['xi']) == relationship
                ind_parent[ind_parent] = checks.flatten()
                self.nodes[node]['data'] = scipy.sparse.coo_matrix(ind_parent)
                return ind_parent


        def impurity_node(y_sigma):
            # impurity of single node
            if self.risk_estimator in ["uPU", "nnPU"]:
                Wp = (y_sigma == 1).sum() * self.pi/n_p
                Wn = (y_sigma == 0).sum()/n_u - Wp

                if Wp + Wn == 0:
                    vstar = float('inf')
                else:
                    vstar = Wp/(Wp + Wn)

            elif self.risk_estimator in ['PN']:
                Wp = (y_sigma == 1).sum() * self.pi/n_p
                Wn = (y_sigma == -1).sum() * (1-self.pi)/n_n

                if Wp + Wn == 0:
                    vstar = float('inf')
                else:
                    vstar = Wp/(Wp + Wn)


            if self.loss == "quadratic":
                if self.risk_estimator == "uPU" and vstar == float('inf'):
                    return -float('inf')
                elif self.risk_estimator == "nnPU" and vstar > 1:
                    return 0
                else:
                    return 4 * (Wp + Wn) * vstar * (1 - vstar)

            elif self.loss == "logistic":
                if self.risk_estimator == "uPU" and vstar > 1:
                    return -float('inf')
                elif self.risk_estimator in ["uPU", "nnPU", "PN"] and vstar in [0,1]:
                    return 0
                elif self.risk_estimator == "nnPU" and vstar > 1:
                    return 0
                else:
                    return (Wp + Wn) * (-vstar*np.log(vstar) - (1-vstar)*np.log(1-vstar))


        def impurity_split(sigma, j, xi):
            mask = (X[sigma, j] <= xi).flatten()
            imT = impurity_node(y[sigma][mask])
            imF = impurity_node(y[sigma][~mask])
            return imT + imF


        def regional_prediction_function(y_sigma):
            if self.risk_estimator in ["uPU", "nnPU"]:
                Wp = (y_sigma == 1).sum() * self.pi/n_p
                Wn = (y_sigma == 0).sum()/n_u - Wp

                if Wp + Wn == 0:
                    vstar = float('inf')
                else:
                    vstar = Wp/(Wp + Wn)

            elif self.risk_estimator in ["PN"]:
                Wp = (y_sigma == 1).sum() * self.pi/n_p
                Wn = (y_sigma == -1).sum() * (1-self.pi)/n_n

                if Wp + Wn == 0:
                    vstar = float('inf')
                else:
                    vstar = Wp/(Wp + Wn)

            if vstar > 0.5:
                return 1
            elif vstar < 0.5:
                return -1
            elif vstar == 0.5:
                return 2*np.random.binomial(1,0.5)-1


        def construct_subtree(node, sigma):
            # first check stopping criteria
            impurity = impurity_node(y[sigma])

            # check node pure
            if self.risk_estimator in ['nnPU', 'PN']:
                c1 = impurity > 0
            elif self.risk_estimator == 'uPU':
                if y[sigma].sum() == 0:
                    c1 = impurity > 0
                else:
                    c1 = impurity > -float('inf')

            # check max depth reached
            if self.max_depth is None:
                c2 = True
            else:
                c2 = node[0] < self.max_depth # max depth reached
            c3 = self.min_samples_leaf < sigma.sum() # minimum samples in node reached
            att_ptp = np.ptp(X[sigma], axis = 0)
            c4 = att_ptp.sum() > 0 # check if there is any variability in features
            # c4 = np.unique(X[sigma], axis = 0).shape[0] > 1
            # check if any of the criteria satisfied
            # if so, turn into a leaf node
            if c1*c2*c3*c4 == 0:
                self.nodes[node]['is_leaf'] = True
                lab = regional_prediction_function(y[sigma])
                self.nodes[node]['g'] = lab
                self.nodes[node]['risk_reduction'] = 0
                self.leaf_count += 1
            else:
                self.nodes[node]['is_leaf'] = False
                # find valid nodes that can be used for a split
                atts = []
                for i in range(self.d):
                    if att_ptp[i] > 0:
                        atts += [i]

                # ranomly pick candiates attributes
                attributes = np.random.choice(atts, size = min(self.max_features, len(atts)), replace = False)
                candidates = []
                candidate_attributes = []
                candidate_cut_points = []
                for i in range(len(attributes)):
                    for j in range(self.max_candidates):
                        # need to guard against errors caused by finite precision
                        a_,b_,c_,d_ = np.unique(X[sigma, attributes[i]])[[0,1,-2,-1]]
                        cut_point = np.random.uniform(a_ + 2*(b_-a_)/5, c_ + 3*(d_-c_)/5)
                        candidates += [[attributes[i], cut_point]]
                        candidate_attributes += [attributes[i]]
                        candidate_cut_points += [cut_point]

                impurities = []
                for i in range(len(candidates)):
                    impurities += [impurity_split(sigma, candidate_attributes[i], candidate_cut_points[i])]

                minimiser = np.argmin(impurities)
                best_attribute = candidate_attributes[minimiser]
                best_cut_point = candidate_cut_points[minimiser]

                self.nodes[node]['j'] = int(best_attribute)
                self.nodes[node]['xi'] = best_cut_point
                self.nodes[node]['risk_reduction'] = impurity - impurities[minimiser]

                # create successors of current node
                self.create_successor(node, 'T')
                self.create_successor(node, 'F')

                # get set of data in these successors
                succs = ((node[0]+1, 2*node[1]), (node[0]+1, 2*node[1]+1))
                sigma_T = data_at_node(succs[0])
                sigma_F = data_at_node(succs[1])

                #keep tabs on how training is going
                # if node[0] > self.current_max_depth:
                    # self.current_max_depth = node[0]
                    # if self.current_max_depth % 30 == 0:
                    #     if self.current_max_depth > 1:
                    #         print('current max depth', self.current_max_depth)
                # print('current max depth', self.current_max_depth)

                # construct_subtree on the sucessors
                construct_subtree(succs[0], sigma_T)
                construct_subtree(succs[1], sigma_F)

        # train the dt
        construct_subtree((0,0), np.ones(n).astype(bool))
        self.is_trained = True
        return self


    def predict(self, X):
        """
        Predict classes for examples in X.
        The predicted class of an input sample is the majority vote by the trees in the forest.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The test samples.

        Returns
        -------
        preds : array of shape (n_samples,)
            The predicted classes.

        """
        # first check to see if the tree is empty/trained
        if self.is_trained:

            preds = np.zeros(len(X)).astype(np.int8)
            for i in range(len(X)):
                X_ = X[i]
                a,b = 0,0
                tnode = self.nodes[(a,b)]
                while not tnode['is_leaf']:
                    check = X_[tnode['j']] <= tnode['xi']

                    if check:
                        b = 2*b
                    else:
                        b = 2*b + 1

                    a += 1
                    tnode = self.nodes[(a,b)]

                if tnode['is_leaf']:
                    preds[i] = tnode['g']

            return preds
        else:
            print('tree not finished training!')


    def n_leaves(self):
        """
        Get the number of leaf nodes in a tree (number of regions created in feature space).

        Returns
        -------
        temp : int
            Number of leaf nodes in the classifier.

        """
        return self.leaf_count

    def get_depth(self):
        """
        Return the depth of the decision tree. The depth of a tree is the maximum distance between the root and any leaf.

        Returns
        -------
        max_depth : int
            The maximum depth of the tree.

        """

        max_depth = -1
        for node in self.nodes.keys():
            if node[0] > max_depth:
                max_depth = node[0]
        return max_depth

    def feature_importances(self):
        """
        Compute the risk reduction feature importances.

        Returns
        -------
        impurities : array-like of shape (n_features,)
            Risk reduction feature importances.

        """

        impurities = np.zeros([self.d])
        for node in self.nodes:
            if self.nodes[node]['j'] is not None:
                impurities[self.nodes[node]['j']] += self.nodes[node]['risk_reduction']

        return impurities


In [2]:
!pip install joblib

In [3]:
# PU ExtraTrees - A Random Forest Classifier for PU Learning
# from tree import PUExtraTree
from joblib import Parallel, delayed
import scipy
import numpy as np

class PUExtraTrees:
    def __init__(self, n_estimators = 100,
                 risk_estimator = 'nnPU',
                 loss = 'quadratic',
                 max_depth = None,
                 min_samples_leaf = 1,
                 max_features = 'sqrt',
                 max_candidates = 1,
                 n_jobs = 1):
        """
        An extra-trees binary classifier that can be trained using only positive and unlabeled samples, or positive and negative samples.

        Parameters
        ----------
        risk_estimator : {"PN", "uPU", "nnPU"}, default='nnPU'
            PU data based risk estimator. Supports supervised (PN) learning, unbiased PU (uPU) learning and nonnegative PU (nnPU) learning.
        loss : {"quadratic", "logistic"}, default='quadratic'
            The function to measure the cost of making an incorrect prediction. Supported loss functions are:
            "quadratic" l(v,y) = (1-vy)^2 and
            "logistic" l(v,y) = ln(1+exp(-vy)).
        max_depth : int or None, default=None
            The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_leaf samples.
        min_samples_leaf : int, default=1
            The minimum number of samples required to be at a leaf node. The default is 1.
        max_features : int or {"sqrt", "all"}, default="sqrt"
            The number of features to consider when looking for the best split. If "sqrt", then max_features = ceil(sqrt(n_features)). If "all", then max_features = n_features.
        max_candidates : int, default=1
            Number of randomly chosen split points to consider for each candidate feature.
        n_jobs : int, default=1
            The number of jobs to run in parallel. fit and predict are all parallelized over the trees.

        Returns
        -------
        None.

        """

        self.n_estimators = n_estimators
        self.risk_estimator = risk_estimator
        self.loss = loss
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.max_candidates = max_candidates
        self.n_jobs = n_jobs

        self.leaf_count = 0
        self.current_max_depth = 0
        self.is_trained = False # indicate if tree empty/trained

    def train_tree(self, P = None, U = None, N = None, pi = None):
        """
        Train a single decision tree.

        Parameters
        ----------
        P : array-like of shape (n_p, n_features), default=None
            Training samples from the positive class.
        U : array-like of shape (n_u, n_features), default=None
            Unlabelled training samples.
        N : array-like of shape (n_n, n_features), default=None
            Training samples from the negative class if performing supervised (PN) learning.
        pi : float
            Prior probability that an example belongs to the positive class.

        Returns
        -------
        g : ET classifier
            An instance of the single tree RF classifier.

        """
        g = PUExtraTree(risk_estimator = self.risk_estimator,
                        loss = self.loss,
                        max_depth = self.max_depth,
                        min_samples_leaf = self.min_samples_leaf,
                        max_features = self.max_features,
                        max_candidates = self.max_candidates)
        g.fit(P = P, U = U, N = N, pi = pi)
        return g

    def predict_tree(self, g, X):
        """
        Predict classes for examples in X using the single DT g.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The test samples.

        Returns
        -------
        preds : array of shape (n_samples,)
            The predicted classes.

        """
        return g.predict(X)

    def fit(self, P = None, U = None, N = None, pi = None):
        """
        Train the random forest.

        Parameters
        ----------
        pi : float
            Prior probability that an example belongs to the positive class.
        P : array-like of shape (n_p, n_features), default=None
            Training samples from the positive class.
        U : array-like of shape (n_u, n_features), default=None
            Unlabeled training samples.
        N : array-like of shape (n_n, n_features), default=None
            Training samples from the negative class if performing PN learning.

        Returns
        -------
        self
            Returns instance of self.

        """
        self.gs = Parallel(n_jobs = min(self.n_jobs, self.n_estimators), prefer="threads")(delayed(self.train_tree)(P = P, U = U, N = N, pi = pi) for i in range(self.n_estimators))
        self.is_trained = True
        return self

    def predict(self, X):
        """
        Predict classes for examples in X.
        The predicted class of an input sample is the majority vote by the trees in the forest.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The test samples.

        Returns
        -------
        preds : array of shape (n_samples,)
            The predicted classes.

        """
        self.preds = Parallel(n_jobs = min(self.n_jobs, self.n_estimators), prefer="threads")(delayed(self.predict_tree)(g, X) for g in self.gs)
        return scipy.stats.mode(np.array(self.preds), axis = 0)[0]

    def n_leaves(self, tree):
        """
        Get the number of leaf nodes in a specified tree

        Parameters
        ----------
        tree : int
            The index of the tree.

        Returns
        -------
        Number of leaf nodes in the specified tree.

        """

        return self.gs[tree].n_leaves()

    def get_depth(self, tree):
        """
        Get the depth of a specified tree in the forest.

        Parameters
        ----------
        tree : int
            The index of the tree.

        Returns
        -------
        Depth of the specified tree.

        """

        return self.gs[tree].get_depth()

    def get_max_depth(self):
        """
        Return the depth of the deepest tree in the forest.

        Returns
        -------
        Maximum depth : int

        """

        depths = []
        for tree in self.gs:
            depths += [tree.get_depth()]
        return np.max(depths)

    def feature_importances(self):
        """
        Get the risk reduction feature importances.

        Returns
        -------
        importances : array of shape (n_features,)
            The risk reduction feature importances.

        """
        importances = np.zeros([self.gs[0].d])
        for tree in self.gs:
            importances += tree.feature_importances()/self.n_estimators

        return importances


In [4]:
!pip install catboost

In [5]:
from catboost.datasets import epsilon
epsilon_train, epsilon_test = epsilon()

No Bottleneck unit testing available.


In [6]:
l = list(epsilon_train[0])

In [7]:
epsilon_train

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,1.0,-0.010828,-0.019600,0.042215,0.003000,0.014533,0.008864,-0.032517,0.011299,0.015190,...,0.036370,0.011610,0.010202,0.029232,-0.016093,0.018768,-0.010783,-0.003415,-0.013111,0.010197
1,-1.0,-0.004703,0.016913,0.006024,-0.011034,0.035253,-0.021374,0.005930,-0.014309,0.039498,...,0.006867,0.002999,0.000582,-0.003425,-0.023347,-0.010011,-0.043020,-0.032248,-0.004244,-0.041520
2,1.0,0.002508,0.016845,-0.010293,-0.015634,0.018036,-0.023563,-0.005037,0.005102,0.037780,...,-0.000240,0.007758,0.016737,0.005199,-0.029202,-0.029026,-0.038255,-0.031005,-0.016839,-0.036780
3,-1.0,-0.000007,0.008525,-0.002561,0.024329,-0.005932,0.001965,-0.026056,-0.002887,-0.012315,...,0.006798,-0.017795,-0.039721,0.035222,0.012496,0.035623,-0.005441,0.018508,0.012023,-0.013321
4,-1.0,-0.009352,0.001053,0.038326,0.006666,-0.005453,0.025037,-0.008886,-0.019346,-0.014963,...,-0.010609,-0.008746,0.040222,-0.011202,0.057171,0.017244,0.015305,-0.019711,0.018008,-0.004310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1.0,-0.013020,0.030704,-0.007066,-0.001714,0.029920,-0.017332,0.019626,-0.016661,0.037513,...,0.000722,0.013792,0.008728,-0.003738,-0.000996,-0.010730,-0.036103,-0.014462,-0.019896,-0.044031
399996,-1.0,0.013424,0.002060,0.007270,0.015533,0.006022,-0.036704,0.014432,0.013070,0.007521,...,0.021567,-0.041164,-0.020147,0.010669,0.028408,0.031195,-0.002247,-0.022245,-0.041987,-0.020384
399997,-1.0,-0.043320,-0.000236,0.027009,0.009198,-0.008228,-0.034195,-0.010594,0.005818,0.013951,...,-0.000732,-0.025494,0.007514,0.030475,-0.015690,-0.017973,-0.016226,0.002462,-0.017922,-0.029265
399998,1.0,-0.011515,0.022920,-0.001483,0.007006,0.031677,-0.041371,0.017700,0.001839,0.042509,...,-0.005267,0.005969,0.014357,-0.021295,-0.010826,-0.024578,-0.039043,-0.035559,-0.012287,-0.034348


In [8]:
y_train = epsilon_train[0]
X = epsilon_train.drop(columns = 0)
n_p = 1000
positive_indices = np.random.choice(np.where(y_train == 1)[0], size = n_p, replace = False)
P = X.iloc[positive_indices]
U = X.copy()

In [9]:
X_test = epsilon_test.drop(columns = 0)
y_test = epsilon_test[0]

In [10]:
X_test

,1,2,3,4,5,6,7,8,9,10,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,0.005439,0.013331,-0.002186,-0.014590,0.015632,-0.032606,-0.004455,0.013611,0.024088,0.010940,...,0.029023,0.028153,-0.001714,-0.048453,-0.030330,-0.006301,-0.022238,-0.009459,0.027544,-0.026216
1,0.001442,0.033820,-0.048836,0.000652,-0.028718,0.013421,-0.006827,0.053082,-0.016931,0.049545,...,-0.016412,0.005543,-0.017588,0.005169,0.037386,-0.002206,0.023466,0.023459,0.036497,0.033899
2,0.004597,-0.042784,-0.004416,-0.005693,0.000731,-0.025873,0.031471,0.059522,0.003261,0.015846,...,-0.020841,-0.030902,0.005387,-0.017727,-0.011851,0.007834,-0.002806,-0.004059,0.024565,-0.001050
3,-0.009080,0.017605,-0.009870,0.007386,0.021338,-0.042682,-0.004471,0.035229,0.037935,-0.005612,...,-0.025915,0.005119,0.022496,-0.005170,-0.023424,-0.026319,-0.036478,-0.036575,-0.002760,-0.021496
4,-0.011689,-0.021413,0.012358,-0.012036,-0.009324,-0.023587,0.007309,0.034027,-0.020042,-0.050917,...,0.006731,0.011447,-0.015999,-0.016919,0.047534,-0.004458,0.013541,0.036077,-0.004313,0.008140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.002185,0.004446,0.022291,0.020920,0.020427,0.012598,0.030112,0.011091,0.001913,-0.020676,...,-0.024602,0.006985,0.016945,0.007586,0.037605,0.028264,-0.001602,0.001870,0.006102,-0.003627
99996,-0.011844,0.020148,-0.026866,0.003328,0.027158,0.005969,0.017370,-0.011501,0.029321,0.006667,...,0.026503,-0.025448,-0.037498,-0.027911,0.010793,-0.004598,-0.025452,-0.013585,-0.004808,-0.020536
99997,0.026138,0.014778,0.001464,-0.026422,0.009453,-0.011432,0.017415,0.048577,0.025200,0.007717,...,0.026015,-0.003136,0.020917,-0.011271,0.040341,-0.020855,-0.020545,-0.022856,0.049474,-0.010935
99998,-0.031459,-0.035492,0.003074,-0.030789,-0.013167,0.002857,0.035517,0.007557,-0.012632,-0.023446,...,0.031911,0.028818,0.015955,-0.005749,0.003205,0.018634,0.010508,0.001254,0.010319,-0.001581


In [11]:
import numpy as np
print(np.unique(y_train))
print(np.unique(y_test))
print(X_test.shape)
print(P.shape)
print(U.shape)

[-1.  1.]
[-1.  1.]
(100000, 2000)
(1000, 2000)
(400000, 2000)


In [12]:
X_test.iloc[8]

1      -0.009143
2       0.013158
3       0.014550
4      -0.036574
5       0.029358
          ...   
1996   -0.011096
1997   -0.031525
1998   -0.028132
1999   -0.007734
2000   -0.037095
Name: 8, Length: 2000, dtype: float32

In [13]:
from sklearn.metrics import f1_score, accuracy_score

In [14]:
g = PUExtraTrees(n_estimators = 100,
                 risk_estimator = 'nnPU',
                 loss = 'quadratic',
                 max_depth = None,
                 min_samples_leaf = 1,
                 max_features = 'sqrt',
                 max_candidates = 1,
                 n_jobs = 4)
g.fit(P=P, U=U, pi = 0.5)

In [16]:
predictions = g.predict(X_test)

KeyError: 0

In [ ]:
predictions.shape

In [ ]:
f = f1_score(predictions, test_labels, average= "macro")
f

In [ ]:
g1 = PUExtraTrees(n_estimators = 100,
                 risk_estimator = 'nnPU',
                 loss = 'logistic',
                 max_depth = None,
                 min_samples_leaf = 1,
                 max_features = 'sqrt',
                 max_candidates = 1,
                 n_jobs = 4)
g1.fit(P=P, U=U, pi = 0.5)

In [ ]:
predictions1 = g1.predict(X_test)
print('Accuracy', (predictions1 == test_labels).mean())

In [ ]:
g2 = PUExtraTrees(n_estimators = 100,
                 risk_estimator = 'uPU',
                 loss = 'logistic',
                 max_depth = None,
                 min_samples_leaf = 1,
                 max_features = 'sqrt',
                 max_candidates = 1,
                 n_jobs = 4)
g2.fit(P=P, U=U, pi = 0.5)

In [ ]:
predictions1 = g2.predict(X_test)
print('Accuracy', (predictions1 == test_labels).mean())